# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 3 Assignment: Creating Columns in Pandas**

**Student Name: Jie Miao**

# Assignment Instructions

For this assignment you will use the **reg-33-spring-2019.csv** dataset.  This is a dataset that I generated specifically for this semester.  You can find the CSV file on my data site, at this location: [reg-33-spring-2019.csv](http://data.heatonresearch.com/data/t81-558/datasets/reg-33-spring-2019.csv).

For this assignment, load and modify the data set.  You will submit this modified dataset to the **submit** function.  See [Assignment #1](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class1.ipynb) for details on how to submit an assignment or check that one was submitted.

Modify the dataset as follows:

* Add a column named *ratio* that is *max* divided by *number*.
* Replace the *cat2* column with dummy variables. 
* Replace the *item* column with an index encoding value. (for example 0 for the first class, 1 for the next, etc. see function *encode_text_index*)
* Remove all other columns.
* Your submitted dataframe will have these columns: *height*,*max*,*number*,*length*,*item*,*ratio*,*cat2-CA-0*,*cat2-CA-1*,*cat2-CA-10*,*cat2-CA-11*,*cat2-CA-12*,*cat2-CA-13*,*cat2-CA-14*,*cat2-CA-15*,*cat2-CA-16*,*cat2-CA-17*,*cat2-CA-18*,*cat2-CA-19*,*cat2-CA-1A*,*cat2-CA-1B*,*cat2-CA-1C*,*cat2-CA-1D*,*cat2-CA-1E*,*cat2-CA-1F*,*cat2-CA-2*,*cat2-CA-20*,*cat2-CA-21*,*cat2-CA-22*,*cat2-CA-23*,*cat2-CA-24*,*cat2-CA-25*,*cat2-CA-26*,*cat2-CA-27*,*cat2-CA-3*,*cat2-CA-4*,*cat2-CA-5*,*cat2-CA-6*,*cat2-CA-7*,*cat2-CA-8*,*cat2-CA-9*,*cat2-CA-A*,*cat2-CA-B*,*cat2-CA-C*,*cat2-CA-D*,*cat2-CA-E*,*cat2-CA-F*.

# Helpful Functions

You will see these at the top of every module and assignment.  These are simply a set of reusable functions that we will make use of.  Each of them will be explained as the semester progresses.  They are explained in greater detail as the course progresses.  Class 4 contains a complete overview of these functions.

In [1]:
import base64
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from sklearn import preprocessing


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = f"{name}-{tv}"
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"


# Regression chart.
def chart_regression(pred, y, sort=True):
    t = pd.DataFrame({'pred': pred, 'y': y.flatten()})
    if sort:
        t.sort_values(by=['y'], inplace=True)
    plt.plot(t['y'].tolist(), label='expected')
    plt.plot(t['pred'].tolist(), label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean())
                          >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
        * (normalized_high - normalized_low) + normalized_low


# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment #3 Sample Code

The following code provides a starting point for this assignment.

In [50]:
import os
import pandas as pd
from scipy.stats import zscore

# This is your student key that I emailed to you at the beginnning of the semester.
key = "qqeAIdN7kdaHWUmLYRyM5aHJxb8Yt5Ssw0qi11wb"  # This is an example key and will not work.

# You must also identify your source file.  (modify for your local setup)
# file='/resources/t81_558_deep_learning/assignment_yourname_class1.ipynb'  # IBM Data Science Workbench
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\t81_558_class1_intro_python.ipynb'  # Windows
file='/Users/jie/Desktop/t81_558_deep_learning-master/assignments/assignment_JieMiao_class3.ipynb' # Mac/Linux
# file = "C:\\Users\\jeffh\\Dropbox\\school\\teaching\\wustl\\classes\\T81_558_deep_learning\\solutions\\assignment_solution_class2.ipynb"

# Begin assignment
path = "/Users/jie/Desktop"

filename_read = os.path.join(path,"reg-33-spring-2019.csv")
df = pd.read_csv(filename_read)

# Add assignment code here
# Add a column named ratio that is max divided by number.
df.insert(1, 'ratio', (df['max'] /df['number']))
column_bef = df.columns
colmn = ['height','max','number','length','item','ratio','cat2']
colmn_del =[]
for col in column_bef:
    if col not in colmn:
        colmn_del.append(col)
# Replace the cat2 column with dummy variables.
encode_text_dummy(df, 'cat2')

        
# Replace the item column with an index encoding value. (for example 0 for the first class, 1 for the next, etc. see function encode_text_index)
encode_text_index(df, 'item')
          
# Remove all other columns.
for item in colmn_del:
    df.drop(item, axis=1, inplace=True)
    
df

#submit(source_file=file,data=df,key=key,no=3)


,ratio,height,max,number,length,item,cat2-CA-0,cat2-CA-1,cat2-CA-10,cat2-CA-11,...,cat2-CA-6,cat2-CA-7,cat2-CA-8,cat2-CA-9,cat2-CA-A,cat2-CA-B,cat2-CA-C,cat2-CA-D,cat2-CA-E,cat2-CA-F
0,2.694043,4284.51,44907,16669,12471.1127,9,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,5.643897,806.88,48831,8652,10035.7085,16,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1.764273,2859.80,40760,23103,14442.6566,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.900283,5823.87,33597,17680,15121.4937,23,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1.236659,5576.05,29848,24136,18093.9147,20,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,3.301515,2030.39,46624,14122,12942.8882,30,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1.866843,6804.32,41022,21974,19150.6343,26,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1.348813,4334.69,32791,24311,16803.2572,29,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2.776903,5865.44,64563,23250,17532.0030,25,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,3.745858,5615.70,50865,13579,14460.4902,13,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
